In [ ]:
%matplotlib inline

# Polynomial regression.

A [PolynomialRegressor][gemseo.mlearning.regression.algos.polyreg.PolynomialRegressor] is a polynomial regression model
based on a [LinearRegressor][gemseo.mlearning.regression.algos.linreg.LinearRegressor].
This design choice was made
because a polynomial regression model is a generalized linear model
whose basis functions are monomials.
Thus,
a [PolynomialRegressor][gemseo.mlearning.regression.algos.polyreg.PolynomialRegressor] benefits
from the same settings as [LinearRegressor][gemseo.mlearning.regression.algos.linreg.LinearRegressor]:
offset can be set to zero and regularization techniques can be used.

!!! info "See also"
    You will find more information about these settings
    in [the example about the linear regression model][linear-regression].


In [ ]:
from __future__ import annotations

from matplotlib import pyplot as plt
from numpy import array

from gemseo import create_design_space
from gemseo import create_discipline
from gemseo import sample_disciplines
from gemseo.mlearning import create_regression_model

## Problem

In this example,
we represent the function $f(x)=(6x-2)^2\sin(12x-4)$
by the [AnalyticDiscipline][gemseo.disciplines.analytic.AnalyticDiscipline].

!!! quote "References"
      Alexander I. J. Forrester, Andras Sobester, and Andy J. Keane.
      Engineering design via surrogate modelling: a practical guide. Wiley, 2008.



In [ ]:
discipline = create_discipline(
    "AnalyticDiscipline",
    name="f",
    expressions={"y": "(6*x-2)**2*sin(12*x-4)"},
)

and seek to approximate it over the input space



In [ ]:
input_space = create_design_space()
input_space.add_variable("x", lower_bound=0.0, upper_bound=1.0)

To do this,
we create a training dataset with 6 equispaced points:



In [ ]:
training_dataset = sample_disciplines(
    [discipline], input_space, "y", algo_name="PYDOE_FULLFACT", n_samples=6
)

## Basics

### Training

Then,
we train a polynomial regression model with a degree of 2 (default)
from these samples:



In [ ]:
model = create_regression_model("PolynomialRegressor", training_dataset)
model.learn()

### Prediction

Once it is built,
we can predict the output value of $f$ at a new input point:



In [ ]:
input_value = {"x": array([0.65])}
output_value = model.predict(input_value)
output_value

as well as its Jacobian value:



In [ ]:
jacobian_value = model.predict_jacobian(input_value)
jacobian_value

### Plotting

Of course,
you can see that the quadratic model is no good at all here:



In [ ]:
test_dataset = sample_disciplines(
    [discipline], input_space, "y", algo_name="PYDOE_FULLFACT", n_samples=100
)
input_data = test_dataset.get_view(variable_names=model.input_names).to_numpy()
reference_output_data = test_dataset.get_view(variable_names="y").to_numpy().ravel()
predicted_output_data = model.predict(input_data).ravel()
plt.plot(input_data.ravel(), reference_output_data, label="Reference")
plt.plot(input_data.ravel(), predicted_output_data, label="Regression - Basics")
plt.grid()
plt.legend()
plt.show()

## Settings

The [PolynomialRegressor][gemseo.mlearning.regression.algos.polyreg.PolynomialRegressor] has many options
defined in the [PolynomialRegressor][gemseo.mlearning.regression.algos.polyreg_settings.PolynomialRegressor_Settings] Pydantic model.
Most of them are presented
in [the example about the linear regression model][polynomial-regression].

The only one we will look at here is the degree of the polynomial regression model.
This information can be set with the `degree` keyword.
For example,
we can use a cubic regression model instead of a quadratic one:



In [ ]:
model = create_regression_model("PolynomialRegressor", training_dataset, degree=3)
model.learn()

and see that this model seems to be better:



In [ ]:
predicted_output_data_ = model.predict(input_data).ravel()
plt.plot(input_data.ravel(), reference_output_data, label="Reference")
plt.plot(input_data.ravel(), predicted_output_data, label="Regression - Basics")
plt.plot(input_data.ravel(), predicted_output_data_, label="Regression - Degree(3)")
plt.grid()
plt.legend()
plt.show()